In [65]:
import io
from math import log
from numpy import array
from numpy import argmax
import torch
import random
from math import log
from numpy import array
from numpy import argmax
import numpy as np
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import Adam
from torchcrf import CRF
from torch.optim.lr_scheduler import ExponentialLR, CyclicLR
from typing import List, Tuple, AnyStr
from tqdm.notebook import tqdm
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
from copy import deepcopy
from datasets import load_dataset, load_metric
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
import heapq
from transformers import AutoModel, AutoTokenizer
import pandas as pd

In [7]:
from Preprocess import Preprocess
from Pipeline import Pipeline
from languages.English import English
from datasets import Dataset
from sklearn.preprocessing import MultiLabelBinarizer

In [82]:
english = English()

preproccesor = Preprocess(english.tokenize, english.clean)
data = Pipeline().get_data(language=english.name, preproccesor=preproccesor)
train_data, validation_data = Pipeline().split_data(data)


Loading data...


In [83]:
train_data

,question_text,document_title,language,document_plaintext,document_url,answer_start,answer_text,is_answerable,tokenized_question,tokenized_plaintext,tokenized_answer,answer_end,cleaned_question,cleaned_plaintext
5415,How many people die from AIDs each year?,AIDS,english,Mother-to-child transmission is another contri...,https://en.wikipedia.org/wiki/Epidemiology%20o...,-1,,False,"[How, many, people, die, from, AIDs, each, yea...","[Mother-to-child, transmission, is, another, c...",[],-1,"[mani, peopl, die, aid, year, ?]","[mother-to-child, transmiss, anoth, contribut,..."
751,What language is spoken in Nepal?,Languages of Nepal,english,"The official language of Nepal is Nepali, form...",https://en.wikipedia.org/wiki/Languages%20of%2...,6,Nepali,True,"[What, language, is, spoken, in, Nepal, ?]","[The, official, language, of, Nepal, is, Nepal...",[Nepali],6,"[languag, spoken, nepal, ?]","[offici, languag, nepal, nepali, ,, former, ca..."
3774,Who is the Mayor of San Francisco?,Mayor of San Francisco,english,The current mayor is former District 5 Supervi...,https://en.wikipedia.org/wiki/Mayor%20of%20San...,15,London Breed,True,"[Who, is, the, Mayor, of, San, Francisco, ?]","[The, current, mayor, is, former, District, 5,...","[London, Breed]",16,"[mayor, san, francisco, ?]","[current, mayor, former, district, 5, supervis..."
7283,What percent of Denmark is Catholic?,Religion in Denmark,english,According to official statistics from January ...,https://en.wikipedia.org/wiki/Religion%20in%20...,-1,,False,"[What, percent, of, Denmark, is, Catholic, ?]","[According, to, official, statistics, from, Ja...",[],-1,"[percent, denmark, cathol, ?]","[accord, offici, statist, januari, 2018, ,, 75..."
4382,What country does the island of Lesbos belong to?,Lesbos,english,"In English and most other European languages, ...",https://en.wikipedia.org/wiki/Lesbos,-1,,False,"[What, country, does, the, island, of, Lesbos,...","[In, English, and, most, other, European, lang...",[],-1,"[countri, island, lesbo, belong, ?]","[english, european, languag, ,, includ, greek,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,Who is head coach of the Rouen Baseball 76?,Huskies de Rouen,english,European Champions Cup: 2-3 (4th Place),https://en.wikipedia.org/wiki/Huskies%20de%20R...,-1,,False,"[Who, is, head, coach, of, the, Rouen, Basebal...","[European, Champions, Cup, :, 2-3, (, 4th, Pla...",[],-1,"[head, coach, rouen, basebal, 76, ?]","[european, champion, cup, :, 2-3, (, 4th, plac..."
7891,When was the first safety pin developed?,Safety pin,english,The laryngologist Dr. Chevalier Jackson devise...,https://en.wikipedia.org/wiki/Safety%20pin,-1,,False,"[When, was, the, first, safety, pin, developed...","[The, laryngologist, Dr., Chevalier, Jackson, ...",[],-1,"[first, safeti, pin, develop, ?]","[laryngologist, dr., chevali, jackson, devis, ..."
4859,When was West Bathurst founded in Australia?,"Bathurst, New South Wales",english,Bathurst is often referred to as the Gold Coun...,https://en.wikipedia.org/wiki/Bathurst%2C%20Ne...,-1,,False,"[When, was, West, Bathurst, founded, in, Austr...","[Bathurst, is, often, referred, to, as, the, G...",[],-1,"[west, bathurst, found, australia, ?]","[bathurst, often, refer, gold, countri, site, ..."
3264,When was the character Sherlock Holmes first i...,Sherlock Holmes,english,First appearing in print in 1887's A Study in ...,https://en.wikipedia.org/wiki/Sherlock%20Holmes,5,1887,True,"[When, was, the, character, Sherlock, Holmes, ...","[First, appearing, in, print, in, 1887, 's, A,...",[1887],5,"[charact, sherlock, holm, first, introduc, ?]","[first, appear, print, 1887, 's, studi, scarle..."


In [84]:
train_data.to_parquet('train_data.parquet')
validation_data.to_parquet('validation_data.parquet')

In [85]:
train_data = pd.read_parquet('train_data.parquet')
validation_data = pd.read_parquet('validation_data.parquet')

In [86]:
train_data

,question_text,document_title,language,document_plaintext,document_url,answer_start,answer_text,is_answerable,tokenized_question,tokenized_plaintext,tokenized_answer,answer_end,cleaned_question,cleaned_plaintext
5415,How many people die from AIDs each year?,AIDS,english,Mother-to-child transmission is another contri...,https://en.wikipedia.org/wiki/Epidemiology%20o...,-1,,False,"[How, many, people, die, from, AIDs, each, yea...","[Mother-to-child, transmission, is, another, c...",[],-1,"[mani, peopl, die, aid, year, ?]","[mother-to-child, transmiss, anoth, contribut,..."
751,What language is spoken in Nepal?,Languages of Nepal,english,"The official language of Nepal is Nepali, form...",https://en.wikipedia.org/wiki/Languages%20of%2...,6,Nepali,True,"[What, language, is, spoken, in, Nepal, ?]","[The, official, language, of, Nepal, is, Nepal...",[Nepali],6,"[languag, spoken, nepal, ?]","[offici, languag, nepal, nepali, ,, former, ca..."
3774,Who is the Mayor of San Francisco?,Mayor of San Francisco,english,The current mayor is former District 5 Supervi...,https://en.wikipedia.org/wiki/Mayor%20of%20San...,15,London Breed,True,"[Who, is, the, Mayor, of, San, Francisco, ?]","[The, current, mayor, is, former, District, 5,...","[London, Breed]",16,"[mayor, san, francisco, ?]","[current, mayor, former, district, 5, supervis..."
7283,What percent of Denmark is Catholic?,Religion in Denmark,english,According to official statistics from January ...,https://en.wikipedia.org/wiki/Religion%20in%20...,-1,,False,"[What, percent, of, Denmark, is, Catholic, ?]","[According, to, official, statistics, from, Ja...",[],-1,"[percent, denmark, cathol, ?]","[accord, offici, statist, januari, 2018, ,, 75..."
4382,What country does the island of Lesbos belong to?,Lesbos,english,"In English and most other European languages, ...",https://en.wikipedia.org/wiki/Lesbos,-1,,False,"[What, country, does, the, island, of, Lesbos,...","[In, English, and, most, other, European, lang...",[],-1,"[countri, island, lesbo, belong, ?]","[english, european, languag, ,, includ, greek,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,Who is head coach of the Rouen Baseball 76?,Huskies de Rouen,english,European Champions Cup: 2-3 (4th Place),https://en.wikipedia.org/wiki/Huskies%20de%20R...,-1,,False,"[Who, is, head, coach, of, the, Rouen, Basebal...","[European, Champions, Cup, :, 2-3, (, 4th, Pla...",[],-1,"[head, coach, rouen, basebal, 76, ?]","[european, champion, cup, :, 2-3, (, 4th, plac..."
7891,When was the first safety pin developed?,Safety pin,english,The laryngologist Dr. Chevalier Jackson devise...,https://en.wikipedia.org/wiki/Safety%20pin,-1,,False,"[When, was, the, first, safety, pin, developed...","[The, laryngologist, Dr., Chevalier, Jackson, ...",[],-1,"[first, safeti, pin, develop, ?]","[laryngologist, dr., chevali, jackson, devis, ..."
4859,When was West Bathurst founded in Australia?,"Bathurst, New South Wales",english,Bathurst is often referred to as the Gold Coun...,https://en.wikipedia.org/wiki/Bathurst%2C%20Ne...,-1,,False,"[When, was, West, Bathurst, founded, in, Austr...","[Bathurst, is, often, referred, to, as, the, G...",[],-1,"[west, bathurst, found, australia, ?]","[bathurst, often, refer, gold, countri, site, ..."
3264,When was the character Sherlock Holmes first i...,Sherlock Holmes,english,First appearing in print in 1887's A Study in ...,https://en.wikipedia.org/wiki/Sherlock%20Holmes,5,1887,True,"[When, was, the, character, Sherlock, Holmes, ...","[First, appearing, in, print, in, 1887, 's, A,...",[1887],5,"[charact, sherlock, holm, first, introduc, ?]","[first, appear, print, 1887, 's, studi, scarle..."


In [73]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=3)

loading configuration file config.json from cache at /Users/christianjensen/.cache/huggingface/hub/models--bert-base-cased/snapshots/a8d257ba9925ef39f3036bfc338acf5283c512d9/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /Users/christianjensen/.cache/huggingface/hub/models--bert-base-cased/snapshots/a8d257ba9925ef39f3036bfc338acf5283c512

In [74]:
def _tag_token(token, index, answer_start, answer_end):
    """Tag a token with the IOB format"""
    if index == answer_start:
        return 0 # B
    elif answer_start < index <= answer_end:
        return 1 # I
    else:
        return 2 # O

def _tag_sentence(sentence, answer_start, answer_end):
    """Tag a sentence with the IOB format"""
    return [
        _tag_token(token, index, answer_start, answer_end)
        for index, token in enumerate(sentence)
    ]

def _convert_to_iob(dataset):
    """Tag the dataset with the IOB format"""
    dataset['tags'] = dataset.apply(lambda row: _tag_sentence(row['tokenized_plaintext'], row['answer_start'], row['answer_end']), axis=1)
    return dataset

def _tokenize_and_align_labels(batch):
    """Realign the labels to the tokenized inputs. This is due to the tokenizer splitting the words into subwords"""
    tokenized_inputs = tokenizer(
                        batch['tokenized_question'],
                        batch['tokenized_plaintext'],
                        max_length=512,
                        padding='max_length',
                        truncation='only_second',
                        is_split_into_words=True,
                        return_tensors='pt'
                        )
    
    labels = []
    for i, label in enumerate(batch["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        sep_index = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            elif sep_index == None: # Set all the tokens of the question to -100
                label_ids.append(-100)
                if tokenized_inputs['input_ids'][i][word_idx] == tokenizer.sep_token_id: # Find the index of the first SEP token to know where the question ends
                    sep_index = word_idx
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx-sep_index]) # The tags are offset by the number of tokens in the question
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    
    tokenized_inputs['labels'] = labels
    return tokenized_inputs


def extract_X(dataset, language: str = ""):
    """Extract features from the dataset"""
    dataset = _convert_to_iob(dataset)

    train_dataset = Dataset.from_pandas(dataset[['tokenized_question', 'tokenized_plaintext', 'tags']])
    train_dataset = train_dataset.map(_tokenize_and_align_labels, batched=True)
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    return train_dataset


def extract_y(dataset, language: str = ""):
    """Extract the labels from the dataset"""
    return None

In [87]:
X_train = extract_X(train_data, 'english')
X_val = extract_X(validation_data, 'english')

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [45]:

args = TrainingArguments(
    "test-bert",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    # push_to_hub=True,
)

In [46]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [47]:
metric = load_metric("seqeval")

In [53]:
X_train

Dataset({
    features: ['tokenized_question', 'tokenized_plaintext', 'tags', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 6700
})

In [59]:
precision_recall_fscore_support(X_train['labels'][0], X_train['labels'][0], average='macro')

(1.0, 1.0, 1.0, None)

In [60]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    binarizer = MultiLabelBinarizer().fit(true_labels)

    true_predictions = binarizer.transform(true_predictions)
    true_labels = binarizer.transform(true_labels)

    P, R, F1, _ = precision_recall_fscore_support(true_labels, true_predictions, average='macro', labels=[0,1,2])
    return {
        "precision": P,
        "recall": R,
        "f1": F1,
    }

In [61]:
trainer = Trainer(
    model,
    args,
    train_dataset=X_train,
    eval_dataset=X_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [62]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: __index_level_0__, tokenized_plaintext, tags, tokenized_question. If __index_level_0__, tokenized_plaintext, tags, tokenized_question are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/opt/homebrew/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6700
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 630
Automatic Weights & Biases logging enabled, to disable set os.environ

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: christian2903. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/630 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/homebrew/lib/python3.9/site-packages/transformers/data/data_collator.py:318: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sequence_length = torch.tensor(batch["input_ids"]).shape[1]
/opt/homebrew/lib/python3.9/site-packages/transformers/data/data_collator.py:329: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()